In [3]:
# -*- coding: utf-8 -*-
"""urdu-to-roman-urdu-translator.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1WnSE_S2eypz48WPQ-Vo8o_WoYMQqIBTJ

### Clone repo & unzip dataset
"""

# Cell 1: Clone repo & unzip dataset
!git clone https://github.com/amir9ume/urdu_ghazals_rekhta.git
!unzip -q urdu_ghazals_rekhta/dataset/dataset.zip -d urdu_ghazals_rekhta/dataset/

from pathlib import Path
root = Path("urdu_ghazals_rekhta/dataset/dataset")
print("Root exists:", root.exists())
print("Example authors (first 8):", [p.name for p in sorted(root.iterdir()) if p.is_dir()][:8])

fatal: destination path 'urdu_ghazals_rekhta' already exists and is not an empty directory.
replace urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/ur/silsile-tod-gayaa-vo-sabhii-jaate-jaate-ahmad-faraz-ghazals? [y]es, [n]o, [A]ll, [N]one, [r]ename: Root exists: True
Example authors (first 8): ['ahmad-faraz', 'akbar-allahabadi', 'allama-iqbal', 'altaf-hussain-hali', 'ameer-khusrau', 'bahadur-shah-zafar', 'dagh-dehlvi', 'fahmida-riaz']


In [4]:
# Cell 2: Build parallel dataset (Urdu -> Roman)
import pandas as pd
from pathlib import Path

root = Path("urdu_ghazals_rekhta/dataset/dataset")
pairs = []
for author in sorted(root.iterdir()):
    if not author.is_dir():
        continue
    ur_dir = author / "ur"
    en_dir = author / "en"
    if not ur_dir.exists() or not en_dir.exists():
        continue
    for ur_file in sorted(ur_dir.iterdir()):
        en_file = en_dir / ur_file.name
        if not en_file.exists():
            continue
        ur_lines = open(ur_file, encoding="utf-8", errors="ignore").read().splitlines()
        en_lines = open(en_file, encoding="utf-8", errors="ignore").read().splitlines()
        for u,e in zip(ur_lines, en_lines):
            if u and e:
                pairs.append((u.strip(), e.strip()))
df = pd.DataFrame(pairs, columns=["urdu","roman"])
print("Collected pairs:", len(df))
df.head()

Collected pairs: 21003


,urdu,roman
0,آنکھ سے دور نہ ہو دل سے اتر جائے گا,aañkh se duur na ho dil se utar jā.egā
1,وقت کا کیا ہے گزرتا ہے گزر جائے گا,vaqt kā kyā hai guzartā hai guzar jā.egā
2,اتنا مانوس نہ ہو خلوت غم سے اپنی,itnā mānūs na ho ḳhalvat-e-ġham se apnī
3,تو کبھی خود کو بھی دیکھے گا تو ڈر جائے گا,tū kabhī ḳhud ko bhī dekhegā to Dar jā.egā
4,ڈوبتے ڈوبتے کشتی کو اچھالا دے دوں,Dūbte Dūbte kashtī ko uchhālā de duuñ


In [5]:

# Cell 3: Normalize roman side and clean Urdu (simple rules)
import re

def normalize_roman(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.replace("<unk>", "")
    # normalize some diacritics to ascii-ish forms
    reps = {"ā":"a","ī":"i","ū":"u","ḳ":"q","ġ":"gh","ñ":"n","ṭ":"t","ḍ":"d","ś":"sh","’":"'", "—":" "}
    for a,b in reps.items():
        text = text.replace(a,b)
    # keep common roman characters, digits, apostrophe and spaces
    text = re.sub(r"[^a-z0-9'\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def clean_urdu(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"\s+", " ", text.strip())
    return text

df["roman"] = df["roman"].apply(normalize_roman)
df["urdu"] = df["urdu"].apply(clean_urdu)
# drop very short / empty lines
df = df[(df["roman"].str.len() > 1) & (df["urdu"].str.len() > 0)].reset_index(drop=True)
print("After cleaning:", len(df))
df.sample(5)

After cleaning: 21003


,urdu,roman
15963,بیاد گرمیٔ صحبت بہ رنگ شعلہ دہکے ہے,bayad e garmi e sohbat ba rang e sho ala dahke...
2167,خودی کی موت ہے اندیشہ ہائے گوناں گوں,qhudi ki maut hai andesha ha e guna gun
12532,تاب کس کو جو حال میرؔ سنے,taab kis ko jo hal e 'mir' sune
6815,زخم پنہاں کی ہے نشانی بھی,zaqhm e pinhan ki hai nishani bhi
3716,کیا تماشا ہے رگ لیلیٰ میں ڈوبا نیشتر,kya tamasha hai rag e laila men duuba neshtar


In [6]:

# Cell 4: Shuffle and split 50/25/25
from sklearn.model_selection import train_test_split
import os
os.makedirs("data", exist_ok=True)

df_shuf = df.sample(frac=1, random_state=42).reset_index(drop=True)
train, tmp = train_test_split(df_shuf, test_size=0.5, random_state=42)
valid, test = train_test_split(tmp, test_size=0.5, random_state=42)

train.to_csv("data/train.tsv", sep="\t", index=False)
valid.to_csv("data/valid.tsv", sep="\t", index=False)
test.to_csv("data/test.tsv", sep="\t", index=False)

print("Saved splits: train/valid/test =", len(train), len(valid), len(test))

Saved splits: train/valid/test = 10501 5251 5251


In [7]:
# Cell 5: Learn BPE from scratch on roman training data
from collections import Counter, defaultdict
import json
import os

# Read roman training lines (space-separated words)
train_roman_lines = train["roman"].tolist()

# Prepare corpus as list of tokenized words represented as tuple of symbols ending with </w>
def get_tokenized_corpus(lines):
    corpus = []
    for line in lines:
        for word in line.strip().split():
            # represent a word as list of characters with end-of-word marker
            symbols = list(word) + ["</w>"]
            corpus.append(tuple(symbols))
    return corpus

corpus = get_tokenized_corpus(train_roman_lines)

# BPE helpers
def get_stats(corpus):
    pairs = defaultdict(int)
    for word in corpus:
        prev = word[0]
        for ch in word[1:]:
            pairs[(prev,ch)] += 1
            prev = ch
    return pairs

def merge_pair(pair, corpus):
    # pair is a tuple (a,b); merge occurrences in corpus
    a,b = pair
    merged = a + b
    new_corpus = []
    for word in corpus:
        new_word = []
        i = 0
        while i < len(word):
            # if pair matches at position i
            if i < len(word)-1 and word[i] == a and word[i+1] == b:
                new_word.append(merged)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_corpus.append(tuple(new_word))
    return new_corpus

def learn_bpe(corpus, num_merges=2000, verbose=True):
    corpus = list(corpus)
    merges = []
    for i in range(num_merges):
        pairs = get_stats(corpus)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        merges.append(best)
        corpus = merge_pair(best, corpus)
        if verbose and (i+1) % 100 == 0:
            print(f"Merges learned: {i+1}")
    return merges, corpus

# Learn merges (you can adjust num_merges for experiments; suggested 1500-2500)
N_MERGES = 2000  # Adjustable for experiments
print("Learning BPE merges (this may take a while)...")
bpe_merges, final_corpus = learn_bpe(corpus, num_merges=N_MERGES, verbose=True)
print("Learned merges:", len(bpe_merges))

# Build token vocabulary (unique tokens in final_corpus)
tokens = set()
for word in final_corpus:
    tokens.update(word)
# ensure common special tokens
specials = ["<pad>", "<sos>", "<eos>", "<unk>"]
vocab_tokens = specials + sorted(list(tokens))
bpe_token_to_id = {tok: idx for idx, tok in enumerate(vocab_tokens)}
bpe_id_to_token = {idx: tok for tok, idx in bpe_token_to_id.items()}

# Save merges and vocab
os.makedirs("data/bpe_from_scratch", exist_ok=True)
with open("data/bpe_from_scratch/merges.json", "w", encoding="utf-8") as f:
    json.dump([list(m) for m in bpe_merges], f, ensure_ascii=False, indent=2)
with open("data/bpe_from_scratch/vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_tokens, f, ensure_ascii=False, indent=2)

print("BPE vocab size:", len(bpe_token_to_id))

# BPE encode function that applies merges learned above
def bpe_encode_word(word, merges):
    # initial symbols
    symbols = list(word) + ["</w>"]
    symbols = symbols.copy()
    merges_set = {tuple(m): i for i,m in enumerate(merges)}  # map to rank for tie-break
    while True:
        pairs = [(symbols[i], symbols[i+1]) for i in range(len(symbols)-1)]
        # find candidate pairs present in merges
        candidates = [(p, merges_set.get(p, None)) for p in pairs]
        # choose leftmost pair that appears earliest in merges (i.e., smallest index)
        best_pair, best_rank = None, None
        for p, r in candidates:
            if r is not None:
                if best_rank is None or r < best_rank:
                    best_pair, best_rank = p, r
        if best_pair is None:
            break
        # merge all occurrences of best_pair
        a,b = best_pair
        merged = a+b
        new_symbols = []
        i = 0
        while i < len(symbols):
            if i < len(symbols)-1 and symbols[i] == a and symbols[i+1] == b:
                new_symbols.append(merged)
                i += 2
            else:
                new_symbols.append(symbols[i])
                i += 1
        symbols = new_symbols
    return symbols  # list of BPE tokens (with possible merged tokens and </w>)

def bpe_encode_text(text, merges=bpe_merges):
    # encodes a full line -> list of tokens (strings)
    toks = []
    for word in text.strip().split():
        w_toks = bpe_encode_word(word, merges)
        toks.extend(w_toks)
    return toks

# quick example
sample = train_roman_lines[0] if train_roman_lines else "example"
print("Example roman line:", sample)
print("BPE encoded tokens (first 30):", bpe_encode_text(sample)[:30])

Learning BPE merges (this may take a while)...
Merges learned: 100
Merges learned: 200
Merges learned: 300
Merges learned: 400
Merges learned: 500
Merges learned: 600
Merges learned: 700
Merges learned: 800
Merges learned: 900
Merges learned: 1000
Merges learned: 1100
Merges learned: 1200
Merges learned: 1300
Merges learned: 1400
Merges learned: 1500
Merges learned: 1600
Merges learned: 1700
Merges learned: 1800
Merges learned: 1900
Merges learned: 2000
Learned merges: 2000
BPE vocab size: 1981
Example roman line: chaman zangar hai a ina e bad e bahari ka
BPE encoded tokens (first 30): ['chaman</w>', 'za', 'n', 'gar</w>', 'hai</w>', 'a</w>', 'ina</w>', 'e</w>', 'bad</w>', 'e</w>', 'baha', 'ri</w>', 'ka</w>']


In [8]:
# Cell 6: Urdu character-level vocabulary
src_vocab = {"<pad>":0, "<sos>":1, "<eos>":2, "<unk>":3}
for line in train["urdu"]:
    for ch in line:
        if ch not in src_vocab:
            src_vocab[ch] = len(src_vocab)

print("Urdu vocab size:", len(src_vocab))
# invert mapping for decoding if needed
src_id_to_char = {i:c for c,i in src_vocab.items()}

Urdu vocab size: 62


In [9]:

# Cell 7: Dataset & DataLoaders (uses our bpe_encode_text & src_vocab)
import torch
from torch.utils.data import Dataset, DataLoader

PAD_IDX_SRC = src_vocab["<pad>"]
PAD_IDX_TGT = bpe_token_to_id["<pad>"]
SOS_ID = bpe_token_to_id["<sos>"]
EOS_ID = bpe_token_to_id["<eos>"]
UNK_ID = bpe_token_to_id["<unk>"]

MAX_SRC_LEN = 80
MAX_TGT_LEN = 80

class GhazalDataset(Dataset):
    def __init__(self, df):
        self.src_lines = df["urdu"].tolist()
        self.tgt_lines = df["roman"].tolist()
    def __len__(self):
        return len(self.src_lines)
    def __getitem__(self, idx):
        return self.src_lines[idx], self.tgt_lines[idx]

def encode_src_line(line):
    ids = [src_vocab.get(ch, src_vocab["<unk>"]) for ch in list(line)]
    ids = [src_vocab["<sos>"]] + ids[:MAX_SRC_LEN-2] + [src_vocab["<eos>"]]
    return ids

def encode_tgt_line(line):
    toks = bpe_encode_text(line)  # list of token strings
    ids = [bpe_token_to_id.get(t, UNK_ID) for t in toks]
    ids = [SOS_ID] + ids[:MAX_TGT_LEN-2] + [EOS_ID]
    return ids

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    src_lens, tgt_lens = [], []
    for s,t in batch:
        src_ids = torch.tensor(encode_src_line(s), dtype=torch.long)
        tgt_ids = torch.tensor(encode_tgt_line(t), dtype=torch.long)
        src_batch.append(src_ids); tgt_batch.append(tgt_ids)
        src_lens.append(src_ids.size(0)); tgt_lens.append(tgt_ids.size(0))
    src_pad = torch.nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX_SRC)
    tgt_pad = torch.nn.utils.rnn.pad_sequence(tgt_batch, batch_first=True, padding_value=PAD_IDX_TGT)
    return src_pad, torch.tensor(src_lens, dtype=torch.long), tgt_pad, torch.tensor(tgt_lens, dtype=torch.long)

# Adjustable for experiments
BATCH_SIZE = 64  # Suggested: 32, 64, 128

train_ds = GhazalDataset(train)
valid_ds = GhazalDataset(valid)
test_ds = GhazalDataset(test)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=1, pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=1, pin_memory=True)

# quick check
batch = next(iter(train_dl))
print("Batch shapes src/tgt:", batch[0].shape, batch[2].shape)

Batch shapes src/tgt: torch.Size([64, 46]) torch.Size([64, 19])


In [17]:
# Cell 8: Model definitions without Attention (Vanilla BiLSTM Encoder-Decoder)
import torch.nn as nn
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# Adjustable hyperparameters for experiments
EMB_DIM = 256  # Suggested: 128, 256, 512
ENC_HID = 256 # Suggested: 256, 512
DEC_HID = 256  # Suggested: 256, 512
ENC_LAYERS = 2  # Suggested: 1, 2, 3, 4
DEC_LAYERS = 4  # Suggested: 2, 3, 4
DROPOUT = 0.3  # Suggested: 0.1, 0.3, 0.5

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim=EMB_DIM, enc_hidden=ENC_HID, n_layers=ENC_LAYERS, dropout=DROPOUT, pad_idx=PAD_IDX_SRC):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(emb_dim, enc_hidden, num_layers=n_layers,
                           bidirectional=True, batch_first=True,
                           dropout=dropout if n_layers > 1 else 0.0)
        self.dropout = nn.Dropout(dropout)
        self.enc_hidden = enc_hidden

    def forward(self, src, src_lens):
        embedded = self.dropout(self.embedding(src))
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, src_lens.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, (h_n, c_n) = self.rnn(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        return out, (h_n, c_n)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim=EMB_DIM, dec_hidden=DEC_HID, n_layers=DEC_LAYERS, dropout=DROPOUT, pad_idx=PAD_IDX_TGT):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(emb_dim, dec_hidden,
                           num_layers=n_layers, batch_first=True,
                           dropout=dropout if n_layers > 1 else 0.0)
        self.fc_out = nn.Linear(dec_hidden + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.dec_hidden = dec_hidden
        self.n_layers = n_layers

    def forward(self, input_step, hidden):
        emb = self.dropout(self.embedding(input_step).unsqueeze(1))  # [B,1,emb]
        out, hidden = self.rnn(emb, hidden)
        out = out.squeeze(1)
        emb = emb.squeeze(1)
        pred = self.fc_out(torch.cat((out, emb), dim=1))  # [B,V]
        return pred, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_hidden=ENC_HID, dec_hidden=DEC_HID):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.h_proj = nn.Linear(enc_hidden*2, dec_hidden)
        self.c_proj = nn.Linear(enc_hidden*2, dec_hidden)

    def forward(self, src, src_lens, tgt, teacher_forcing_ratio=0.5):
        B = src.size(0)
        T = tgt.size(1)
        V = self.decoder.fc_out.out_features
        outputs = torch.zeros(B, T, V).to(src.device)

        enc_out, (h_n, c_n) = self.encoder(src, src_lens)

        top_fwd, top_bwd = h_n[-2,:,:], h_n[-1,:,:]
        top_c_fwd, top_c_bwd = c_n[-2,:,:], c_n[-1,:,:]
        h0 = torch.tanh(self.h_proj(torch.cat((top_fwd, top_bwd), dim=1)))
        c0 = torch.tanh(self.c_proj(torch.cat((top_c_fwd, top_c_bwd), dim=1)))
        h0 = h0.unsqueeze(0).repeat(self.decoder.n_layers,1,1).contiguous()
        c0 = c0.unsqueeze(0).repeat(self.decoder.n_layers,1,1).contiguous()
        dec_hidden = (h0, c0)

        input_tok = tgt[:,0]
        for t in range(1, T):
            logits, dec_hidden = self.decoder(input_tok, dec_hidden)
            outputs[:,t,:] = logits
            teacher_force = (torch.rand(1).item() < teacher_forcing_ratio)
            top1 = logits.argmax(1)
            input_tok = tgt[:,t] if teacher_force else top1
        return outputs

# Instantiate model
enc = Encoder(len(src_vocab), emb_dim=EMB_DIM, enc_hidden=ENC_HID, n_layers=ENC_LAYERS, dropout=DROPOUT).to(DEVICE)
dec = Decoder(len(bpe_token_to_id), emb_dim=EMB_DIM, dec_hidden=DEC_HID, n_layers=DEC_LAYERS, dropout=DROPOUT).to(DEVICE)
model = Seq2Seq(enc, dec).to(DEVICE)
print("Vanilla Model ready. #params:", sum(p.numel() for p in model.parameters()))

Using device: cuda
Vanilla Model ready. #params: 6536893


In [18]:
# Cell 9: Training Loop (50 epochs max, early stopping, timing in seconds, train/val accuracy + loss + ppl)
import torch.optim as optim
import time
from torch.nn.utils import clip_grad_norm_
import math

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX_TGT)

# Adjustable for experiments
LR = 5e-4  # Suggested: 1e-3, 5e-4, 1e-4
optimizer = optim.Adam(model.parameters(), lr=LR)

N_EPOCHS = 50
CLIP = 1.0
PATIENCE = 5   # stop if no improvement for 5 consecutive epochs

def train_epoch(model, dataloader, optimizer, criterion, clip=CLIP):
    model.train()
    epoch_loss, correct, total = 0, 0, 0
    for src, src_lens, tgt, tgt_lens in dataloader:
        src, src_lens, tgt = src.to(DEVICE), src_lens.to(DEVICE), tgt.to(DEVICE)
        optimizer.zero_grad()
        output = model(src, src_lens, tgt, teacher_forcing_ratio=0.5)
        output_dim = output.shape[-1]
        logits = output[:, 1:].reshape(-1, output_dim)
        targets = tgt[:, 1:].reshape(-1)
        loss = criterion(logits, targets)
        loss.backward()
        clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        preds = logits.argmax(1)
        mask = targets != PAD_IDX_TGT
        correct += (preds.eq(targets) & mask).sum().item()
        total += mask.sum().item()
    acc = correct / total if total > 0 else 0
    return epoch_loss / len(dataloader), acc

def evaluate(model, dataloader, criterion):
    model.eval()
    epoch_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for src, src_lens, tgt, tgt_lens in dataloader:
            src, src_lens, tgt = src.to(DEVICE), src_lens.to(DEVICE), tgt.to(DEVICE)
            output = model(src, src_lens, tgt, teacher_forcing_ratio=0.0)
            output_dim = output.shape[-1]
            logits = output[:, 1:].reshape(-1, output_dim)
            targets = tgt[:, 1:].reshape(-1)
            loss = criterion(logits, targets)
            epoch_loss += loss.item()
            preds = logits.argmax(1)
            mask = targets != PAD_IDX_TGT
            correct += (preds.eq(targets) & mask).sum().item()
            total += mask.sum().item()
    acc = correct / total if total > 0 else 0
    return epoch_loss / len(dataloader), acc

best_valid_loss = float("inf")
patience_counter = 0

for epoch in range(1, N_EPOCHS+1):
    start_time = time.time()
    train_loss, train_acc = train_epoch(model, train_dl, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_dl, criterion)
    valid_ppl = math.exp(valid_loss)
    elapsed = time.time() - start_time

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "best_seq2seq.pt")
        patience_counter = 0
        print(f"{epoch}: Train Loss={train_loss:.3f},Train Acc={train_acc*100:.2f}%, "
              f"Valid Loss={valid_loss:.3f},Valid Acc={valid_acc*100:.2f}%, Valid PPL={valid_ppl:.2f}, "
              f"Time={elapsed:.2f}s  <-- saved")
    else:
        patience_counter += 1
        print(f"{epoch}: Train Loss={train_loss:.3f},Train Acc={train_acc*100:.2f}%, "
              f"Valid Loss={valid_loss:.3f},Valid Acc={valid_acc*100:.2f}%, Valid PPL={valid_ppl:.2f}, "
              f"Time={elapsed:.2f}s  (no improvement {patience_counter}/{PATIENCE})")

    if patience_counter >= PATIENCE:
        print("Early stopping triggered.")
        break


1: Train Loss=6.252,Train Acc=9.84%, Valid Loss=6.069,Valid Acc=10.86%, Valid PPL=432.09, Time=51.52s  <-- saved
2: Train Loss=5.961,Train Acc=11.42%, Valid Loss=6.060,Valid Acc=10.97%, Valid PPL=428.46, Time=52.39s  <-- saved
3: Train Loss=5.881,Train Acc=11.68%, Valid Loss=6.061,Valid Acc=10.86%, Valid PPL=428.93, Time=51.63s  (no improvement 1/5)
4: Train Loss=5.806,Train Acc=11.92%, Valid Loss=6.062,Valid Acc=10.97%, Valid PPL=429.12, Time=51.75s  (no improvement 2/5)
5: Train Loss=5.754,Train Acc=12.14%, Valid Loss=6.062,Valid Acc=10.86%, Valid PPL=429.26, Time=51.78s  (no improvement 3/5)
6: Train Loss=5.707,Train Acc=12.34%, Valid Loss=6.067,Valid Acc=10.86%, Valid PPL=431.18, Time=51.96s  (no improvement 4/5)
7: Train Loss=5.664,Train Acc=12.65%, Valid Loss=6.069,Valid Acc=10.86%, Valid PPL=432.16, Time=52.69s  (no improvement 5/5)
Early stopping triggered.


In [12]:
# Cell 10: Evaluation (BLEU, Perplexity, CER) with Vanilla model

# --- Install dependencies if missing ---
try:
    import sacrebleu
except ImportError:
    !pip install sacrebleu --quiet
    import sacrebleu

try:
    import editdistance
except ImportError:
    !pip install editdistance --quiet
    import editdistance

import numpy as np

# --- BPE Decode Function ---
def bpe_decode(tokens):
    words, current_word = [], ""
    for tok in tokens:
        if tok.endswith("</w>"):
            current_word += tok.replace("</w>", "")
            words.append(current_word)
            current_word = ""
        else:
            current_word += tok
    if current_word:
        words.append(current_word)
    return " ".join(words)

# --- Metrics Computation ---
def compute_metrics(model, dataloader, criterion):
    model.eval()
    refs, hyps = [], []
    test_loss = 0
    num_batches = 0
    with torch.no_grad():
        for src, src_lens, tgt, tgt_lens in dataloader:
            src, src_lens, tgt = src.to(DEVICE), src_lens.to(DEVICE), tgt.to(DEVICE)
            output = model(src, src_lens, tgt, teacher_forcing_ratio=0.0)
            output_dim = output.shape[-1]
            logits = output[:, 1:].reshape(-1, output_dim)
            targets = tgt[:, 1:].reshape(-1)
            loss = criterion(logits, targets)
            test_loss += loss.item()
            num_batches += 1
            preds = output.argmax(-1).cpu().numpy()
            for i in range(len(src)):
                # decode prediction
                pred_tokens = [bpe_id_to_token[idx] for idx in preds[i] if idx in bpe_id_to_token]
                if "<eos>" in pred_tokens:
                    pred_tokens = pred_tokens[1:pred_tokens.index("<eos>")]
                else:
                    pred_tokens = pred_tokens[1:]
                hyp = bpe_decode(pred_tokens)

                # decode reference
                ref_tokens = [bpe_id_to_token[idx.item()] for idx in tgt[i] if idx.item() in bpe_id_to_token]
                if "<eos>" in ref_tokens:
                    ref_tokens = ref_tokens[1:ref_tokens.index("<eos>")]
                else:
                    ref_tokens = ref_tokens[1:]
                ref = bpe_decode(ref_tokens)

                refs.append(ref)
                hyps.append(hyp)

    bleu = sacrebleu.corpus_bleu(hyps, [refs])
    cer = np.mean([
        editdistance.eval(h, r) / len(r) if len(r) > 0 else 0
        for h, r in zip(hyps, refs)
    ])
    test_loss /= num_batches
    test_ppl = math.exp(test_loss)
    return {"BLEU": bleu.score, "PPL": test_ppl, "CER": cer}

# --- Load Best Model + Evaluate on test set ---
model.load_state_dict(torch.load("best_seq2seq.pt", map_location=DEVICE))
metrics = compute_metrics(model, test_dl, criterion)
print("Test metrics:", metrics)

Test metrics: {'BLEU': 0.0069640264920121115, 'PPL': 444.2004792065532, 'CER': np.float64(0.77819266987962)}


In [13]:
# Cell 11: Inference with Beam Search (adapted for vanilla)

def translate_sentence(model, urdu_text, max_len=MAX_TGT_LEN, beam_size=3):
    model.eval()
    with torch.no_grad():
        src_ids = torch.tensor([encode_src_line(urdu_text)], dtype=torch.long).to(DEVICE)
        src_lens = torch.tensor([src_ids.size(1)], dtype=torch.long).to(DEVICE)
        enc_out, (h_n, c_n) = model.encoder(src_ids, src_lens)
        top_fwd, top_bwd = h_n[-2,:,:], h_n[-1,:,:]
        top_c_fwd, top_c_bwd = c_n[-2,:,:], c_n[-1,:,:]
        h0 = torch.tanh(model.h_proj(torch.cat((top_fwd, top_bwd), dim=1)))
        c0 = torch.tanh(model.c_proj(torch.cat((top_c_fwd, top_c_bwd), dim=1)))
        h0 = h0.unsqueeze(0).repeat(model.decoder.n_layers,1,1).contiguous()
        c0 = c0.unsqueeze(0).repeat(model.decoder.n_layers,1,1).contiguous()
        initial_hidden = (h0, c0)

        # Beam: (tokens, hidden, score)
        beams = [([SOS_ID], initial_hidden, 0.0)]
        for _ in range(max_len):
            new_beams = []
            for seq, hidden, score in beams:
                input_tok = torch.tensor([seq[-1]], dtype=torch.long).to(DEVICE)
                logits, hidden = model.decoder(input_tok, hidden)
                log_probs = torch.log_softmax(logits, dim=1)
                topk = torch.topk(log_probs, beam_size)
                for idx, log_prob in zip(topk.indices[0], topk.values[0]):
                    new_seq = seq + [idx.item()]
                    new_score = score + log_prob.item()
                    new_beams.append((new_seq, hidden, new_score))
            beams = sorted(new_beams, key=lambda x: x[2], reverse=True)[:beam_size]
            if all(seq[-1] == EOS_ID for seq,_,_ in beams):
                break
        best_seq = beams[0][0][1:]  # skip <sos>
        tokens = [bpe_id_to_token.get(idx, "<unk>") for idx in best_seq]
        if "<eos>" in tokens:
            tokens = tokens[:tokens.index("<eos>")]
        return bpe_decode(tokens)

# Example test sentence
sample = test.sample(1).iloc[0]
sample_urdu = sample["urdu"]
ref_roman   = sample["roman"]
pred_roman  = translate_sentence(model, sample_urdu)

print("Input Urdu:", sample_urdu)
print("Reference Roman:", ref_roman)
print("Predicted Roman:", pred_roman)

# Save the model after training
torch.save(model.state_dict(), "urdu_to_roman_urdu_model.pth")

Input Urdu: وہ رات ہے کوئی ذرہ بھی محو خواب نہیں
Reference Roman: vo raat hai koi zarra bhi mahv e qhvab nahin
Predicted Roman: vo e e e e e e e
